### 이 문서는 비전공 초보가 제작하였으므로, 틀린 부분이 있을지도 모릅니다.

### 설치 및 설정은 아래 페이지를 참고하여 주세요.
- https://github.com/bage79/nlp4kor/blob/master/INSTALL.md

#### 다양한 유틸리티 클래스가 배포되었습니다.
- https://github.com/bage79/nlp4kor/tree/master/bage_utils
- 차후 강좌에서 계속 사용될 예정이오니, 아래에 import 되는 클래스들은 미리 한번 보시면 이해하시는데, 큰 도움이 되실 것입니다.

# FFNN for 한글 띄어쓰기

<img src="img/word_spacing.jpg">

#### 소스 코드: https://github.com/hunkim/DeepLearningZeroToAll/blob/master/lab-09-2-xor-nn.py

### 한글 문자(음절) 데이터를 입력하기 위해 변환하려면? (text -> vector)

### (방법1) 1d-array one-hot-vector (predefined dictionary)
- 자음, 모음, 완성형, 영어/숫자/특수문자 의 모든 문자를 one hot vector 로 표시
- vector 종류 수 = 11,316
- 미리 vector 변환을 생성해 둘 수 있으나, vector의 크기가 매우 큼. 또한 한자등 다른 문자들은 제외됨.

In [1]:
from bage_utils.hangul_util import HangulUtil # 한글처리
from bage_utils.num_util import NumUtil # 숫자(int, str) 처리

In [2]:
print('자음:', 'len:', len(HangulUtil.JA_LIST), HangulUtil.JA_LIST)
print('모음:', 'len:', len(HangulUtil.MO_LIST), HangulUtil.MO_LIST)
print('완성형:', 'len:', len(HangulUtil.WANSUNG_LIST), 
      HangulUtil.WANSUNG_LIST[:10], '...', HangulUtil.WANSUNG_LIST[-10:])
print('전체 한글(자음+모음+완성형)', 'len:', NumUtil.comma_str(len(HangulUtil.HANGUL_LIST)))

자음: len: 30 ('ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ')
모음: len: 21 ('ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ')
완성형: len: 11172 ('가', '각', '갂', '갃', '간', '갅', '갆', '갇', '갈', '갉') ... ('힚', '힛', '힜', '힝', '힞', '힟', '힠', '힡', '힢', '힣')
전체 한글(자음+모음+완성형) len: 11,223


In [3]:
print('전체 한글(자음+모음+완성형) + 키보드특수문자:', 'len:', NumUtil.comma_str(len(HangulUtil.CHAR_LIST)))
print('len(one-hot-vector):', NumUtil.comma_str(len(HangulUtil.to_one_hot_vector('ㄱ'))))
print('ㄱ', HangulUtil.to_one_hot_index('ㄱ'), HangulUtil.to_one_hot_vector('ㄱ'))
print('ㅏ', HangulUtil.to_one_hot_index('ㅏ'), HangulUtil.to_one_hot_vector('ㅏ'))
print('가', HangulUtil.to_one_hot_index('가'), HangulUtil.to_one_hot_vector('가'))
print('힣', HangulUtil.to_one_hot_index('힣'), HangulUtil.to_one_hot_vector('힣'))
print('A', HangulUtil.to_one_hot_index('A'), HangulUtil.to_one_hot_vector('A'))
print('a', HangulUtil.to_one_hot_index('a'), HangulUtil.to_one_hot_vector('a'))
print('0', HangulUtil.to_one_hot_index('0'), HangulUtil.to_one_hot_vector('0'))
print('?', HangulUtil.to_one_hot_index('?'), HangulUtil.to_one_hot_vector('?'))

전체 한글(자음+모음+완성형) + 키보드특수문자: len: 11,317
len(one-hot-vector): 11,317
ㄱ 0 [1 0 0 ..., 0 0 0]
ㅏ 30 [0 0 0 ..., 0 0 0]
가 51 [0 0 0 ..., 0 0 0]
힣 11222 [0 0 0 ..., 0 0 0]
A 11249 [0 0 0 ..., 0 0 0]
a 11223 [0 0 0 ..., 0 0 0]
0 11275 [0 0 0 ..., 0 0 0]
? 11316 [0 0 0 ..., 0 0 1]


### (방법2) 1d-array non-one-hot-vector
- 초성, 중성, 종성, 기타 순서로 3개의 vector를 생성한 후, 1d-array로 concate.
- 한글이 아닌 경우에는 중성, 종성 부분은 항상 0.
- 경우의 수 = (초성개수*중성개수*종성개수) + 영어개수 + 숫자개수 + 특수문자개수 + ....
- 문자 종류별로 범위를 정의해 주어야 함.

In [4]:
print('초성:', 'len:', len(HangulUtil.CHO_LIST), HangulUtil.CHO_LIST)
print('중성:', 'len:', len(HangulUtil.JUNG_LIST), HangulUtil.JUNG_LIST)
print('종성:', 'len:', len(HangulUtil.JONG_LIST), HangulUtil.JONG_LIST)
print('영어:', 'len:', len(HangulUtil.ENGLISH_LIST), HangulUtil.ENGLISH_LIST)
print('숫자:', 'len:', len(HangulUtil.NUM_LIST), HangulUtil.NUM_LIST)
print('특수문자:', 'len:', len(HangulUtil.KEYBOARD_SPECIAL_LIST), HangulUtil.KEYBOARD_SPECIAL_LIST)

초성: len: 19 ('ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ')
중성: len: 21 ('ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ')
종성: len: 28 ('', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ')
영어: len: 52 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
숫자: len: 10 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
특수문자: len: 32 ['`', '~', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+', '-', '+', '[', ']', '\\', ';', "'", ',', '.', '/', '{', '}', '|', ':', '"', '<', '>', '?']


In [5]:
print('len:', len(HangulUtil.to_cho_jung_jong_vector('?')))
print(HangulUtil.to_cho_jung_jong_vector('ㄱ'))
print(HangulUtil.to_cho_jung_jong_vector('a'))
print(HangulUtil.to_cho_jung_jong_vector('1'))
print(HangulUtil.to_cho_jung_jong_vector('각'))
print(HangulUtil.to_cho_jung_jong_vector('?'))

len: 162
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

### (방법3)  1d-array one-hot-vector (from corpus)
- 한자 및 다른 문자들도 처리하고 싶고, 한글의 모든 문자를 항상 사용하는 것이 아님.
- 따라서, 적당히 큰 말뭉치(corpus)에서 실제 사용되는 음절을 추출하여 사용.
- 이 발표자료의 FFNN에서는 이 방법으로 사용

### 학습용 말뭉치

In [6]:
# 총 문장 수 (dump corpus (ko.wikipedia.org) 문장 단위로 분리. 총 문장: 7,601,655 / 총 문서: 518,062)
!ls -lh ~/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.sentences.gz
!zcat ~/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.sentences.gz | wc -l # Ubuntu
# !gzcat ~/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.sentences.gz | wc -l # OSX

-rw-rw-r-- 1 bage bage 204M  6월  2 11:58 /home/bage/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.sentences.gz
7601655


In [7]:
!zcat ~/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.sentences.gz | head # Ubuntu (Broken pipe error on only Jupyter)
# !gzcat -cd ~/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.sentences.gz | head # OSX

!!는 문장부호의 하나로서 겹느낌표라고 부른다.
!!는 체스에서 '훌륭한 이동'을 했을 때를 나타낸다.
!!는 수학에서 이중 팩토리얼을 나타낸다.
!!! (칙칙칙으로 읽음)은 1996년 결성된 미국의 댄스 펑크 밴드이다.
"Me and Giuliani Down by the Schoolyard (a true story)"나 "Must Be the Moon"로 알려졌다.
새크라멘토 출신이지만 지금은 뉴욕 시, 새크라멘토, 포틀랜드에 기반을 두고 있다.
2006년 레드 핫 칠리 페퍼스의 존 프루시안테가 LA에서 이들의 공연을 보고 영국 순회 공연에 이들을 서포트 밴드로 초대하기도 했다.
《! -attention-》은 1998년 9월 30일 발매한 20th Century의 2번째 앨범이다.
전 곡이 어두운 곡으로 선택되었고 초회한정반, 통상반으로 2가지 형태로 발매되었다.
초회한정반 특전으로 20th Century 오리지날 키홀더를 봉입했다.

gzip: stdout: Broken pipe


In [8]:
!ls -lh ~/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.characters
!cat ~/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.characters | wc -l 
!cat ~/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.characters | head

-rw-rw-r-- 1 bage bage 67K  6월  2 11:58 /home/bage/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.characters
17382


 
!
"
#
$
%
&
'


In [9]:
!grep --color=never 한 ~/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.characters
!grep --color=never 뷁 ~/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.characters
!grep --color=never 쀓 ~/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.characters # not found in corpus

한
뷁


## character (from corpus) -> Vector 
x_data -> y_data => 로 표시

<img src="./img/FFNN for word-spacing.001.jpeg">
<img src="./img/FFNN for word-spacing.002.jpeg">
<img src="./img/FFNN for word-spacing.003.jpeg">

## 전처리 (text_preprocessing.py)

In [10]:
import numpy as np
import os

from bage_utils.base_util import is_my_pc
from bage_utils.datafile_util import DataFileUtil
from bage_utils.dataset import DataSet
from bage_utils.file_util import FileUtil
from bage_utils.hangul_util import HangulUtil
from bage_utils.mongodb_util import MongodbUtil
from bage_utils.num_util import NumUtil
from bage_utils.one_hot_vector import OneHotVector
from nlp4kor.config import log, KO_WIKIPEDIA_ORG_SENTENCES_FILE, KO_WIKIPEDIA_ORG_DATA_DIR

#### 입력데이터(문장) 파일 생성 (Mongodb -> file)

In [11]:
sentences_file = KO_WIKIPEDIA_ORG_SENTENCES_FILE
log.info('sentences_file: %s' % sentences_file)
if not os.path.exists(sentences_file):
    TextPreprocess.dump_corpus(MONGO_URL, db_name='parsed', collection_name='ko.wikipedia.org', sentences_file=sentences_file,
                               mongo_query={})  # mongodb -> text file(corpus)

sentences_file: /home/bage/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.sentences.gz


#### 벡터매핑사전(음절) 파일 생성 (문장-> 음절)

In [12]:
characters_file = os.path.join(KO_WIKIPEDIA_ORG_DATA_DIR, 'ko.wikipedia.org.characters')
log.info('characters_file: %s' % characters_file)
if not os.path.exists(characters_file):
    log.info('collect characters...')
    TextPreprocess.collect_characters(sentences_file, characters_file)  # text file -> characters(unique features)
    log.info('collect characters OK.')

characters_file: /home/bage/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.characters


### One hot vector?

In [13]:
unary_vector = OneHotVector([0])
binary_vector = OneHotVector([0, 1])
ternary_vector = OneHotVector([0, 1, 2])
print('%6s\t%6s\t%6s\t%6s' % ('', 'unary', 'binary', 'ternary'))
for i in [0, 1, 2]:
    print('%6s\t%6s\t%6s\t%6s' % (i, unary_vector.to_vector(i), binary_vector.to_vector(i), ternary_vector.to_vector(i)))

      	 unary	binary	ternary
     0	   [0]	   [0]	[1 0 0]
     1	   [0]	   [1]	[0 1 0]
     2	   [0]	   [0]	[0 0 1]


### One hot vector for 음절

In [14]:
log.info('load characters list...')
features_vector = OneHotVector(DataFileUtil.read_list(characters_file))
labels_vector = OneHotVector([0, 1])  # 붙여쓰기=0, 띄어쓰기=1
log.info('load characters list OK. len: %s' % NumUtil.comma_str(len(features_vector))) # 데이터셋 마다 character 구성과 개수는 다름.

print(features_vector, len(features_vector))
print(labels_vector, len(labels_vector))

load characters list...
load characters list OK. len: 17,380
<bage_utils.one_hot_vector.OneHotVector object at 0x7f0903b872e8> 17380
<bage_utils.one_hot_vector.OneHotVector object at 0x7f0903b7f470> 2


In [15]:
for c in ['a', 'b', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', '가', '각']:
    v = features_vector.to_vector(c) # one hot vector
    _c = features_vector.to_value(v) # character for check
    i = features_vector.to_index(c) # index number in characters list (0~17380)
    print(c, v, _c, i)

a [0 0 0 ..., 0 0 0] a 64
b [0 0 0 ..., 0 0 0] b 65
ㄱ [0 0 0 ..., 0 0 0] ㄱ 3129
ㄲ [0 0 0 ..., 0 0 0] ㄲ 3130
ㄳ [0 0 0 ..., 0 0 0] ㄳ 3131
ㄴ [0 0 0 ..., 0 0 0] ㄴ 3132
가 [0 0 0 ..., 0 0 0] 가 13891
각 [0 0 0 ..., 0 0 0] 각 13892


### dataset (training data, test data, validation data)
- 너무 커서 공유하기 힘드네요.. 가장 큰 파일이 약 26GB
- 직접 생성하시면 됩니다. ^^ (sentences, characters 파일 이용)

##### ko.wikipedia.org.dataset.문장수.left_gram.right_gram.종류.gz
- train: text, int 형식 (one-hot-vector로 저장하면 파일이 너무 커짐)
- test, validation: one-hot-vector 형식

In [16]:
!ls -lh ~/workspace/nlp4kor-ko.wikipedia.org/datasets/

total 38G
-rw-rw-r-- 1 bage bage 2.6G  6월  2 01:21 ko.wikipedia.org.dataset.1000000.left=2.right=2.test.gz
-rw-rw-r-- 1 bage bage 3.2G  6월  2 01:21 ko.wikipedia.org.dataset.1000000.left=2.right=2.train.gz
-rw-rw-r-- 1 bage bage 266M  6월  2 01:21 ko.wikipedia.org.dataset.1000000.left=2.right=2.validation.gz
-rw-rw-r-- 1 bage bage 2.6G  6월  1 11:20 ko.wikipedia.org.dataset.10000.left=2.right=2.test.gz
-rw-rw-r-- 1 bage bage  31M  6월  1 11:20 ko.wikipedia.org.dataset.10000.left=2.right=2.train.gz
-rw-rw-r-- 1 bage bage 266M  6월  1 11:20 ko.wikipedia.org.dataset.10000.left=2.right=2.validation.gz
-rw-rw-r-- 1 bage bage  93M  6월  1 11:01 ko.wikipedia.org.dataset.100.left=2.right=2.test.gz
-rw-rw-r-- 1 bage bage 415K  6월  1 11:00 ko.wikipedia.org.dataset.100.left=2.right=2.train.gz
-rw-rw-r-- 1 bage bage  93M  6월  1 11:00 ko.wikipedia.org.dataset.100.left=2.right=2.validation.gz
-rw-rw-r-- 1 bage bage 2.6G  6월  1 12:43 ko.wikipedia.org.dataset.7601655.left=2.right=2.test.gz
-rw-rw-r-- 1 bage

### tensorflow model files

In [17]:
!ls -lh ~/workspace/nlp4kor-ko.wikipedia.org/models/

total 20K
drwxrwxr-x 2 bage bage 4.0K  6월  2 22:00 word_spacing_model.sentences=1000000.layers=4
drwxrwxr-x 2 bage bage 4.0K  6월  2 21:58 word_spacing_model.sentences=10000.layers=2
drwxrwxr-x 2 bage bage 4.0K  6월  2 21:58 word_spacing_model.sentences=10000.layers=4
drwxrwxr-x 2 bage bage 4.0K  6월  2 21:57 word_spacing_model.sentences=100.layers=2
drwxrwxr-x 2 bage bage 4.0K  6월  3 21:04 word_spacing_model.sentences=100.layers=4


In [18]:
!ls -lh ~/workspace/nlp4kor-ko.wikipedia.org/models/*

/home/bage/workspace/nlp4kor-ko.wikipedia.org/models/word_spacing_model.sentences=1000000.layers=4:
total 80M
-rw-rw-r-- 1 bage bage 253  6월  2 22:00 checkpoint
-rw-rw-r-- 1 bage bage 80M  6월  2 22:00 model.data-00000-of-00001
-rw-rw-r-- 1 bage bage 863  6월  2 22:00 model.index
-rw-rw-r-- 1 bage bage 65K  6월  2 22:00 model.meta

/home/bage/workspace/nlp4kor-ko.wikipedia.org/models/word_spacing_model.sentences=10000.layers=2:
total 80M
-rw-rw-r-- 1 bage bage 251  6월  2 17:10 checkpoint
-rw-rw-r-- 1 bage bage 80M  6월  2 17:10 model.data-00000-of-00001
-rw-rw-r-- 1 bage bage 508  6월  2 17:10 model.index
-rw-rw-r-- 1 bage bage 42K  6월  2 17:10 model.meta

/home/bage/workspace/nlp4kor-ko.wikipedia.org/models/word_spacing_model.sentences=10000.layers=4:
total 80M
-rw-rw-r-- 1 bage bage 251  6월  2 16:07 checkpoint
-rw-rw-r-- 1 bage bage 80M  6월  2 16:07 model.data-00000-of-00001
-rw-rw-r-- 1 bage bage 863  6월  2 16:07 model.index
-rw-rw-r-- 1 bage bage 65K  6월  2 16:07 model.meta

/home/bage/

### Tip. 제 노트북에 GPU가 없어요. GPU 있는 PC에서 개발해야 하나요?
- Pycharm Deployment & Remote Interpreter

## 각종 설정값 (word_spacing.py)

In [19]:
import gzip
import math
import os
import sys
import traceback

import numpy as np
import tensorflow as tf

from bage_utils.base_util import is_my_pc
from bage_utils.datafile_util import DataFileUtil
from bage_utils.dataset import DataSet
from bage_utils.datasets import DataSets
from bage_utils.num_util import NumUtil
from bage_utils.one_hot_vector import OneHotVector
from bage_utils.watch_util import WatchUtil
from nlp4kor.config import log, KO_WIKIPEDIA_ORG_DATA_DIR, KO_WIKIPEDIA_ORG_SENTENCES_FILE

In [20]:
if len(sys.argv) == 2:
    max_sentences = int(sys.argv[1])
else:
    max_sentences = int('1,000,000'.replace(',', '')) if is_my_pc() else int('1,000,000'.replace(',', ''))  # run 100 or 1M data (학습: 17시간 소요)
# max_sentences = 100 if is_my_pc() else FileUtil.count_lines(sentences_file, gzip_format=True) # run 100 or full data (학습시간: 5일 소요)
layers = 4
model_file = os.path.join(KO_WIKIPEDIA_ORG_DATA_DIR, 'models',
                          'word_spacing_model.sentences=%s.layers=%s/model' % (max_sentences, layers))  # .%s' % max_sentences
log.info('max_sentences: %s' % max_sentences)
log.info('layers: %s' % layers)
log.info('model_file: %s' % model_file)

sentences_file = KO_WIKIPEDIA_ORG_SENTENCES_FILE
log.info('sentences_file: %s' % sentences_file)

characters_file = os.path.join(KO_WIKIPEDIA_ORG_DATA_DIR, 'ko.wikipedia.org.characters')
log.info('characters_file: %s' % characters_file)

batch_size = 1000  # mini batch size
left_gram, right_gram = 2, 2
ngram = left_gram + right_gram
log.info('batch_size: %s' % batch_size)
log.info('left_gram: %s, right_gram: %s' % (left_gram, right_gram))
log.info('ngram: %s' % ngram)

features_vector = OneHotVector(DataFileUtil.read_list(characters_file))
labels_vector = OneHotVector([0, 1])  # 붙여쓰기=0, 띄어쓰기=1
n_features = len(features_vector) * ngram  # number of features = 17,380 * 4
n_classes = len(labels_vector) if len(labels_vector) >= 3 else 1  # number of classes = 2 but len=1
log.info('features_vector: %s' % features_vector)
log.info('labels_vector: %s' % labels_vector)
log.info('n_features: %s' % n_features)
log.info('n_classes: %s' % n_classes)

n_hidden1 = 100
learning_rate = 0.01  # 0.1 ~ 0.001
log.info('n_hidden1: %s' % n_hidden1)
log.info('learning_rate: %s' % learning_rate)

max_sentences: 1000000
layers: 4
model_file: /home/bage/workspace/nlp4kor-ko.wikipedia.org/models/word_spacing_model.sentences=1000000.layers=4/model
sentences_file: /home/bage/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.sentences.gz
characters_file: /home/bage/workspace/nlp4kor-ko.wikipedia.org/ko.wikipedia.org.characters
batch_size: 1000
left_gram: 2, right_gram: 2
ngram: 4
features_vector: <bage_utils.one_hot_vector.OneHotVector object at 0x7f0903a48ba8>
labels_vector: <bage_utils.one_hot_vector.OneHotVector object at 0x7f0903b87278>
n_features: 69520
n_classes: 1
n_hidden1: 100
learning_rate: 0.01


### test with samples. (word_spacing.py)

In [21]:
from nlp4kor.ws.word_spacing import WordSpacing
log.info('sample testing...')
test_set = ['예쁜 운동화', '즐거운 동화', '삼풍동 화재']
for s in test_set:
    features, labels = WordSpacing.sentence2features_labels(s, left_gram=1, right_gram=1)
    log.info('%s -> %s' % (features, labels))
    log.info('in : "%s"' % s)
    log.info('out: "%s"' % WordSpacing.spacing(s.replace(' ', ''), labels))
log.info('sample testing OK.\n')

sample testing...
['예쁜', '쁜운', '운동', '동화'] -> [0, 1, 0, 0]
in : "예쁜 운동화"
out: "예쁜 운동화"
['즐거', '거운', '운동', '동화'] -> [0, 0, 1, 0]
in : "즐거운 동화"
out: "즐거운 동화"
['삼풍', '풍동', '동화', '화재'] -> [0, 0, 1, 0]
in : "삼풍동 화재"
out: "삼풍동 화재"
sample testing OK.



### 입력데이터 생성, 학습, 평가 (word_spacing.py)
- WordSpacing.learning() in word_spacing.py
- <u>내용이 너무 길어 실제 소스로 설명합니다.</u>

In [22]:
if not os.path.exists(model_file + '.index') or not os.path.exists(model_file + '.meta'):
    WordSpacing.learning(sentences_file, batch_size, left_gram, right_gram, model_file, features_vector, labels_vector, n_hidden1=n_hidden1, max_sentences=max_sentences, learning_rate=learning_rate, layers=layers)

### 그리고... 
- 지금까지 평가한 것은 4음절 단위에 대한 결과입니다. 하지만 우리가 원하는 것은 문장을 입력으로 했을 때의 결과죠.
- 새로운 문장으로 제대로 띄어쓰기가 되는지 확인해 봅니다.

In [24]:
watch = WatchUtil()
watch.start('check')
log.info('chek result...')
sentences = ['아버지가 방에 들어 가신다.', '가는 말이 고와야 오는 말이 곱다.']
max_test_sentences = 100
with gzip.open(sentences_file, 'rt') as f:
    for line in f:
        if len(sentences) >= max_test_sentences:
            break
        sentences.append(line.strip())
log.info('len(sentences): %s' % NumUtil.comma_str(len(sentences)))

accuracies, sims = [], []
with tf.Session() as sess:
    graph = WordSpacing.build_FFNN(n_features, n_classes, n_hidden1, learning_rate, layers=layers)
    X, Y, predicted, accuracy = graph['X'], graph['Y'], graph['predicted'], graph['accuracy']

    saver = tf.train.Saver()
    try:
        restored = saver.restore(sess, model_file)
    except:
        log.error('restore failed. model_file: %s' % model_file)
    try:
        for i, s in enumerate(sentences):
            log.info('')
            log.info('[%s] in : "%s"' % (i, s))
            features, labels = WordSpacing.sentence2features_labels(s, left_gram, right_gram)
            dataset = DataSet(features=features, labels=labels, features_vector=features_vector, labels_vector=labels_vector)
            dataset.convert_to_one_hot_vector()
            if len(dataset) > 0:
                _predicted, _accuracy = sess.run([predicted, accuracy], feed_dict={X: dataset.features, Y: dataset.labels})  # Accuracy report

                generated_sentence = WordSpacing.spacing(s.replace(' ', ''), _predicted)
                sim, correct, total = WordSpacing.sim_two_sentence(s, generated_sentence, left_gram=left_gram, right_gram=right_gram)

                accuracies.append(_accuracy)
                sims.append(sim)

                log.info('[%s] out: "%s" (accuracy: %.1f%%, sim: %.1f%%=%s/%s)' % (i, generated_sentence, _accuracy * 100, sim * 100, correct, total))
    except:
        log.error(traceback.format_exc())

log.info('chek result OK.')
log.info('mean(accuracy): %.2f%%, mean(sim): %.2f%%' % (np.mean(accuracies)*100, np.mean(sims)*100))
log.info('secs/sentence: %.4f' % (watch.elapsed('check') / len(sentences)))
log.info(watch.summary())

chek result...
len(sentences): 100

build_FFNN(layers=4)
INFO:tensorflow:Restoring parameters from /home/bage/workspace/nlp4kor-ko.wikipedia.org/models/word_spacing_model.sentences=1000000.layers=4/model
Restoring parameters from /home/bage/workspace/nlp4kor-ko.wikipedia.org/models/word_spacing_model.sentences=1000000.layers=4/model

[0] in : "아버지가 방에 들어 가신다."
[0] out: "아버지가 방에 들어가 신다." (accuracy: 81.8%, sim: 66.7%=2/3)

[1] in : "가는 말이 고와야 오는 말이 곱다."
[1] out: "가는 말이고와야오는 말이 곱다." (accuracy: 84.6%, sim: 60.0%=3/5)

[2] in : "!!는 문장부호의 하나로서 겹느낌표라고 부른다."
[2] out: "!!는 문장 부호의 하나로서 겹느낌표라고 부른다." (accuracy: 95.2%, sim: 100.0%=4/4)

[3] in : "!!는 체스에서 '훌륭한 이동'을 했을 때를 나타낸다."
[3] out: "!!는 체스에서 '훌륭한 이동'을 했을 때를 나타낸다." (accuracy: 100.0%, sim: 100.0%=6/6)

[4] in : "!!는 수학에서 이중 팩토리얼을 나타낸다."
[4] out: "!!는 수학에서 이중 팩토리얼을 나타낸다." (accuracy: 100.0%, sim: 100.0%=4/4)

[5] in : "!!! (칙칙칙으로 읽음)은 1996년 결성된 미국의 댄스 펑크 밴드이다."
[5] out: "!!! (칙칙칙으로 읽음)은 1996년 결성된 미국의 댄스 펑크 밴드이다." (accuracy: 100.0%, sim: 100.0%=8/

### Tip. 입력 데이터 증가, 파라미터 조절, 레이어 증가... 성능을 높이려면, 뭐 부터 해야 할까요?
- 여러 가지 해보려면 시간이 적게 걸리는 것부터 해야 겠죠.
- 파라미터 조절 or 레이어 증가 -> 입력 데이터 증가